In [1]:
import matplotlib.pyplot as plt
from scipy.stats import binom
import numpy as np
import math
import scipy.stats as stats
# to use hedgehog, one needs to install two packages vose and hedgehog
# pip install git+https://github.com/MaxHalford/vose
# pip install git+https://github.com/MaxHalford/hedgehog
# you may also need to install graphviz to plot PGM
# conda install -c conda-forge python-graphviz 
import hedgehog as hh
import pandas as pd
from scipy.special import logsumexp
from IPython.display import Markdown as md
def hide_code_in_slideshow():   
    from IPython import display
    import binascii
    import os
    uid = binascii.hexlify(os.urandom(8)).decode()    
    html = """<div id="%s"></div>
    <script type="text/javascript">
        $(function(){
            var p = $("#%s");
            if (p.length==0) return;
            while (!p.hasClass("cell")) {
                p=p.parent();
                if (p.prop("tagName") =="body") return;
            }
            var cell = p;
            cell.find(".input").addClass("hide-in-slideshow")
        });
    </script>""" % (uid, uid)
    display.display_html(html, raw=True)
#  a hack to hide code from cell: https://github.com/damianavila/RISE/issues/32    

In [2]:
%%html
<style>
 .container.slides .celltoolbar, .container.slides .hide-in-slideshow {
    display: None ! important;
}
    
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
th, td {
  padding: 5px;
}
th {
  text-align: left;
}
</style>

In [3]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

# Foreword 

The following code are used for this lecture
  * most of the code have been adapted from https://github.com/aimacode/aima-python

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]



def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

In [5]:
FIFOQueue = deque

LIFOQueue = list

StackAgenda = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [6]:
class RouteProblem(Problem):
    """A problem to find a route between locations on a `Map`.
    Create a problem with RouteProblem(start, goal, map=Map(...)}).
    States are the vertexes in the Map graph; actions are destination states."""
    
    def actions(self, state): 
        """The places neighboring `state`."""
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """Go to the `action` place, if the map says that is possible."""
        return action if action in self.map.neighbors[state] else state
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""
        return self.map.distances[s, s1]
    
    def h(self, node):
        "Straight-line distance between state and the goal."
        locs = self.map.locations
        return straight_line_distance(locs[node.state], locs[self.goal])
    
    
def straight_line_distance(A, B):
    "Straight-line distance between two points."
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5

In [7]:
class Map:
    """A map of places in a 2D world: a graph with vertexes and links between them. 
    In `Map(links, locations)`, `links` can be either [(v1, v2)...] pairs, 
    or a {(v1, v2): distance...} dict. Optional `locations` can be {v1: (x, y)} 
    If `directed=False` then for every (v1, v2) link, we add a (v2, v1) link."""

    def __init__(self, links, locations=None, directed=False):
        if not hasattr(links, 'items'): # Distances are 1 by default
            links = {link: 1 for link in links}
        if not directed:
            for (v1, v2) in list(links):
                links[v2, v1] = links[v1, v2]
        self.distances = links
        self.neighbors = multimap(links)
        self.locations = locations or defaultdict(lambda: (0, 0))

        
def multimap(pairs) -> dict:
    "Given (key, val) pairs, make a dict of {key: [val,...]}."
    result = defaultdict(list)
    for key, val in pairs:
        result[key].append(val)
    return result

In [8]:
# Some specific RouteProblems

romania = Map(
    {('O', 'Z'):  71, ('O', 'S'): 151, ('A', 'Z'): 75, ('A', 'S'): 140, ('A', 'T'): 118, 
     ('L', 'T'): 111, ('L', 'M'):  70, ('D', 'M'): 75, ('C', 'D'): 120, ('C', 'R'): 146, 
     ('C', 'P'): 138, ('R', 'S'):  80, ('F', 'S'): 99, ('B', 'F'): 211, ('B', 'P'): 101, 
     ('B', 'G'):  90, ('B', 'U'):  85, ('H', 'U'): 98, ('E', 'H'):  86, ('U', 'V'): 142, 
     ('I', 'V'):  92, ('I', 'N'):  87, ('P', 'R'): 97},
    {'A': ( 76, 497), 'B': (400, 327), 'C': (246, 285), 'D': (160, 296), 'E': (558, 294), 
     'F': (285, 460), 'G': (368, 257), 'H': (548, 355), 'I': (488, 535), 'L': (162, 379),
     'M': (160, 343), 'N': (407, 561), 'O': (117, 580), 'P': (311, 372), 'R': (227, 412),
     'S': (187, 463), 'T': ( 83, 414), 'U': (471, 363), 'V': (535, 473), 'Z': (92, 539)})


r0 = RouteProblem('A', 'A', map=romania)
# r1 = RouteProblem('A', 'B', map=romania)
# r2 = RouteProblem('N', 'L', map=romania)
# r3 = RouteProblem('E', 'T', map=romania)
# r4 = RouteProblem('O', 'M', map=romania)

# CS5010 Artificial Intelligence Principles
### Lecture 16-17 Uninformed Search 

Lei Fang

University of St Andrews

# Today

* Searching problem in general

* Performance measurements of a searching algorithm

* Two searching algorithms
  * Depth First search (DFS)
  * Breath First search (BFS)
  * Performance analysis on them

# Searching 


* Uninformed search
  * e.g. you are dropped to Mars and need to find the base-station
    * no extra information/heuristic at all
    * the best you can do is to blindly try all options available to you


* Informed search also called *heuristic search* (next week's topic)
  * e.g. from St Andrews to London  (assume you know  a tiny bit about the UK's geography)
    * Aberdeen or Shetland are probably not good choices for your next stop! you are making some *informed* search
    * Edinburgh makes better sense (as you know it is closer to London)

# A searching problem: route finding

Find the route from a starting point (Arad) to a destination (Bucharest)

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romania.png" width = "900" height="400"/></center>

The problems can be distilled as the following ingredients:
* a set of possible **states** that are reachable: for exmaple, ``Arad``, ``Sibiu``
* The **initial state** to start with; for example `Arad`
* A set of one or more **goal states**; for example `Bucharest`

* The **actions** and **transition model** at a given state. 
  * for example, `ACTIONS(Arad) = {ToSibiu, ToTimisoara, ToZerind}`
  * `RESULT(Arad, ToSibiu) = Sibiu`

* **Action cost function**, denoted by `ACTION-COST(s, a, s')`, that returns the cost of the action
  * need both start $s$, $a$ and ending state $s'$ because there might be multiple actions between two states
    * for example, both train or bus are available to travel from Edinburgh to London
  * problem dependent, could be miles, time, ticket price  

* A **path** or **solution**: a set of states interleaved with actions that start with initial and end with the goal state
  * for example: Arad - Sibliu - Fagaras - Bucharest
  * **optimal solution**: solution with the lowest path cost

# Abstraction


Abstraction is the process of removing irrelevant or trivial details of the problem domain

A searching problem is abstracted as : **states**, **initial state**, **goal state**, **actions and transitions**, 
**action cost function**
  

All other details are not included 
  * e.g. the weather when the agent travels; road between states might be blocked/delayed/closed

Abstraction is a mathematical model: `all models are wrong but some are useful`
  * it provides us with the essentials to solve the problem

# State space graph

The abstraction is a collection of: **states**, **initial state**, **goal state**, **actions and transitions**, 
**action cost function**

The collection can also be explicitly represented as a **state space graph**, like the Romania map 
  * **states** are the nodes; we may want to denote initial and goal states with double circle 
  * **state space**: the set of all possible states, i.e. the number of nodes in the graph
  * **edges** are the **actions** between nodes
  * weighted edges denote the **action cost**
  
  
<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romania.png" width = "800" height="400"/></center>  


However, in general it is not **feasible** to do so
  * state space size too large ! 
    * graph would be too large to draw in one go! usually exponentially large 
  * or real-time searching problem (transition models not available apriori)

Graph is usually created on the fly while we do the searching

# Another searching problem: 8 puzzle 

You are allowed to slide the blank tile to its adjacent tile to reach the final state

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/eightpuzzle.png" width = "600" height="400"/></center>



We can find the same ingredients:

* The **initial state** and **goal/destination state** are above
* State space: $9!$ possible states! too many to plot everything in a graph like Romamia map example
  * blank tile has 9 choices, then the rest 1 to 8 have $8!$ possible choices, 
  * together we have $9 \times 8! = 9!$ states
* **Action and transitions**: move the blank tile, e.g.

<center><img src="https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz3.png" width = "600" height="400"/></center>

* **action cost function**: each transition cost the same; uniform cost
* **optimal solution**: a path with the least number of transitions

Again, a lot of other trivial details are not modelled
  * e.g. tile gets stuck etc
  
  
Thanks to abstraction, **8 puzzle** and **route finding** are the same 
  * the same ingredients in the abstraction
  * one algorithm can deal with both of them

# Searching algorithm performance measures


We measure a searching algorithm by the following measures

* **completeness**: if there exists a solution to the search problem, is the algorithm guaranteed to find it
* **optimality**: is the algorithm guaranteed to find the optimal solution 

* **time complexity** how long does the algorithm take to find a solution ?
* **space complexity** how much memory is needed to perform the search ?

* we will use big-O notation to measure complexity
  * it provides the worst case scenario's performance
  * computer scientists care about the worst possible perfomance: it is a guarantee
  
    


* **branching factor** (written as $b$) is used a lot in complexity analysis
  * the maximum number of possible actions/edges for the searching problem
  * e.g. $b=4$ for the Romania map problem: Sibiu has 4 actions available 
  * $b=4$ for 8-puzzle, up, down, left, right (4 transitions)

# Searching algorithm in general


All searching algorithms we consider in this week (and next week) has the following paradigm

* superimpose a **search tree** over the state space graph
* the **root** of the tree is set as the initial state



For example, our initial tree looks like this
  * initial state is the root

State space graph        |  Superimposed search tree (starting point)
:-------------------------:|:-------------------------:
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romania.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/tree2.png)

# Tree node is different from graph state 

We can expand the tree to expect to find a goal node somewhere down the tree


State space graph        |  Superimposed search tree (starting point)
:-------------------------:|:-------------------------:
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romania.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/tree3_3.png)

<div class="alert alert-block alert-info">
    <b>!!! Note a node in the searching tree (right hand side) is different from the a state in the graph</b> 
</div>


State space graph        |  Superimposed search tree (starting point)
:-------------------------:|:-------------------------:
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romania.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/tree3_3.png)

Each node in the tree actually represent **a partial solution** !
  * e.g. Oradea (left one) actually represents a (partial) solution path: Arad -> Sibiu -> Oradea
    * the red arrow traces back to its parent nodes
  * which is of course different from the other Oradea on the right


Therefore, each tree **node** contains some extra information
  * link to its parent; and cost to reach from its parent
  * check `expand` method and `Node` class in the text book

# Different searching algorithms are different ways to expand the trees

Different searching algorithm basically are 
  * just different strategies to expand the tree
  * different strategies have different pros and cons 

# First search strategy: breath first search

Our first searching strategy is **breath first search (BFS)**: a method that pushes uniformly to all directions of the tree
  * explore the nodes 1 step away from root first 
  * then 2 hops away 
  * 3 hops away
  
  
<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/bfsilla.png" width = "400" height="400"/></center>  

# Breadth First Search Algorithm

An interative algorithm with the help of a First In First Out (FIFO) data structure **queue** (called **agenda** in the algorithm)
  * **initialise**: as a queue with only the root (starting state's node)
  * repeat adding extended paths to the **agenda** (some call it *frontier* or *open list*)
  * to stress a node in the tree represents a path, 
    * I have used `path`: list of nodes of the path to the agenda 
    * the book has actually used a data structure **linked list** to represent the path (node can back trace its parent)
    * just different levels of data structure (abstract data types): Linked list (AIAMA book) vs List (algorithm below)



Pseudo-code of BFS, 
* normally known as BFS tree search (tree is acyclic; so no need to do cycle check)

<!-- <div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
    
step 0: Initialize
   
    agenda = FIFOQueue();
    agenda.push([start]);
    
while agenda is not empty:
    
    1. path = agenda.pop()
    2. if is-path-to-goal(path, goal)
        return path
    3. otherwise extend the current path 
        for each connected node (child node of the ending node of path)
            make a new_path that extends the connected node
            agenda.push(new_path)
fail  
</div> -->

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/bfstreepseudo.png" width = "800" height="400"/></center>  

In [9]:
# Python implementation of Breath first search 
def my_breadth_first_tree_search(problem):
    agenda = FIFOQueue([Node(problem.initial)])
    while agenda:
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
#           no cyclic checking 
            agenda.appendleft(child)
    return failure    

# Why FIFO Queue ?



Invariant condition: at all times, paths ending with shallower nodes are at front of the queue
    <center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/queue.png" width = "600" height="400"/></center>  


Shallower nodes will be popped first
  * so will be goal tested first as well


<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/dequeue.png" width = "800" height="400"/></center>  

Next levels $l+1$ are always pushed to the end of the queue
  * won't be considered until all paths ending with level $l$'s paths have been popped and considered 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/enqueue.png" width = "800" height="400"/></center>  

# An example 

Consider the following state space graph 
  * a $3\times 3$ grid
  * actions: use 0,1,2,3 to denote right, down, left, up (clock-wise) 
  * initial state: A; goal state: G
  
  
<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/grid.png" width = "300" height="400"/></center>    

<font color='blue'><b>Step 0</b></font>: **initialisation:**

    agenda = Queue();
    agenda.push([A])
    
---------------------------------------    
<font color='blue'><b>Agenda status</b></font>: 

|iteration | agenda|
| :-| :-|
|   <font color='blue'><b>0:</b></font>     |    <font color='blue'><b>[A]</b></font>   |





--------------------------------------- 
<font color='blue'><b>Tree view</b></font>: 
* <font color='salmon'><i><b>salmon</b></i></font> colored node: paths in the **agenda** (wait to be extended), e.g. A


<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/grid1.png" width = "800" height="400"/></center>    

<font color='blue'><b>Step 1</b></font>: while loop: check `agenda` is not empty 

    path = agenda.pop()      // path = [A]
    goal test on A           // false; 
                             // goal tested so far: A
    extend path: [A,B] [A,D] // A has two successer: B, D (clock-wise expansion)
    push extended paths to agenda
---------------------------------------    


<font color='blue'><b>Agenda status</b></font>: 

|step | agenda|
| :-| :-|
|  0:     |    ~~[A]~~   |
|   <font color='blue'><b>1:</b></font>     |    <font color='blue'><b>[A,B], [A,D]</b></font> |

--------------------------------------- 
<font color='blue'><b>Tree view</b></font>: 
* <font color='salmon'><i><b>salmon</b></i></font> colored node: paths in the **agenda**, e.g. B,D
* <font color='blue'><i><b>blue</b></i></font> node: **extended** nodes (or nodes have been popped), e.g. A
* <font color='gray'><i><b>blank</b></i></font> node: nodes not yet expanded nor in agenda (or others)

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/grid2.png" width = "800" height="400"/></center>    

<font color='blue'><b>Step 2</b></font>: while loop: check `agenda` is not empty 

    path = agenda.pop()          // path = [A, B], pop the front of the queue
    goal test on B               // B is path's ending state and B is not the goal, 
                                 // goal tested so far A,B
    extend path: [A,B,A] [A,B,C] [A,B,E] // B has three successers: A,C,E
    push extended paths to agenda 
---------------------------------------    


<font color='blue'><b>Agenda status</b></font>: 

|step | agenda|
| :-| :-|
|  0:     |    [A]   |
|  1:     |    ~~[A,B]~~, [A,D]|
|  <font color='blue'><b>2:</b></font>     |     [A,D], <font color='blue'><b>[A,B,A], [A,B,C], [A,B,E]</b></font>|


--------------------------------------- 
<font color='blue'><b>Tree view</b></font>: 
* <font color='salmon'><i><b>salmon</b></i></font> colored node: paths in the **agenda**, e.g. D,A,C,E
* <font color='blue'><i><b>blue</b></i></font> node: **extended** nodes (or nodes have been popped/extended), e.g. A,B
* <font color='gray'><i><b>blank</b></i></font> node: nodes not yet expanded nor in agenda (or others)

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/grid3.png" width = "800" height="400"/></center>    

<font color='blue'><b>Step 3</b></font>: : while loop: check `agenda` is not empty 

    path = agenda.pop()          // path = [A, D], pop the front of the queue
    goal test on D               // D is path's ending state and D is not the goal; 
                                 // goal tested so far A,B,D
    extend path: [A,D,A] [A,D,E] [A,D,G] // B has two successers: A,E,G
    push extended paths to agenda
---------------------------------------    



<font color='blue'><b>Agenda status</b></font>: 



|step | agenda|
| :-| :-|
|  0:     |    [A]   |
|  1:     |    [A,B], [A,D]|
|  2:     |    ~~[A,D]~~, [A,B,A], [A,B,C], [A,B,E]|
|   <font color='blue'><b>3:</b></font>     |     [A,B,A], [A,B,C], [A,B,E], <font color='blue'><b>[A,D,A], [A,D,E], [A,D,G]</b></font>|


--------------------------------------- 
<font color='blue'><b>Tree view</b></font>: 
* <font color='salmon'><i><b>salmon</b></i></font> colored node: paths in the **agenda** yet to be considered
* <font color='blue'><i><b>blue</b></i></font> node: **extended** nodes (or nodes have been popped/extended), 
* <font color='gray'><i><b>blank</b></i></font> node: nodes not yet expanded nor in agenda (or others)

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/grid4.png" width = "800" height="400"/></center>    

<font color='blue'><b>Step 4</b></font>: : while loop: check `agenda` is not empty 

    path = agenda.pop()          // path = [A, B, A], pop the front of the queue
    goal test on A               // A is path's ending state and A is not the goal; 
                                 // goal tested so far A,B,D,A (repeated)
    extend path: [A,B,A,B], [A,B,A,D] // A has two successers: B,D
    push extended paths to agenda
---------------------------------------    


<font color='blue'><b>Agenda status</b></font>: 



|step | agenda|
| :-| :-|
|  0:     |    [A]   |
|  1:     |    [A,B], [A,D]|
|  2:     |    [A,D], [A,B,A], [A,B,C], [A,B,E]|
|  3:     |    ~~[A,B,A]~~, [A,B,C], [A,B,E], [A,D,A], [A,D,E], [A,D,G]|
|   <font color='blue'><b>4:</b></font>     |     [A,B,C], [A,B,E], [A,D,A], [A,D,E], [A,D,G], <font color='blue'><b>[A,B,A,B], [A,B,A,D]</b></font>|


--------------------------------------- 
<font color='blue'><b>Tree view</b></font>: 
* <font color='salmon'><i><b>salmon</b></i></font> colored node: paths in the **agenda**, e.g. D,A,C,E
* <font color='blue'><i><b>blue</b></i></font> node: **extended** nodes (or nodes have been popped/extended), e.g. A,B
* <font color='gray'><i><b>blank</b></i></font> node: nodes not yet expanded nor in agenda (or others)

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/grid5.png" width = "800" height="400"/></center>    

fast forward to step 9

<font color='blue'><b>Step 9</b></font>: : while loop: check `agenda` is not empty 

    path = agenda.pop()          // path = [A, D, G], pop the front of the queue
    goal test on G               // G is the goal! 
                                 // goal tested so far A,B,D,A,C,E,A,E,G
    return path
---------------------------------------    
<font color='blue'><b>Agenda status</b></font>: 



|step | agenda|
| :-| :-|
|   <font color='blue'><b>9:</b></font>     |    ~~[A,D,G]~~, [A,B,A,B], [A,B,A,D], [A,B,C,B], [A,B,C,F] ...|

--------------------------------------- 
<font color='blue'><b>Tree view</b></font>: 
* <font color='salmon'><i><b>salmon</b></i></font> colored node: paths in the **agenda**, e.g. D,A,C,E
* <font color='blue'><i><b>blue</b></i></font> node: **extended** nodes (or nodes have been popped/extended), e.g. A,B
* <font color='gray'><i><b>blank</b></i></font> node: nodes not yet expanded nor in agenda (or others)

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/grid6.png" width = "800" height="400"/></center>   

# Performance analysis of BFS


**Complete**: if there is a solution, guaranteed to find it or not ?
  * Yes! assume we have enough memory to store the agenda, the solution path will eventually be popped and checked



**Optimal**: is the solution found optimal ?
  * Yes for this question. Guaranteed to return the shallowest node 
    * or in other words if costs are uniform; the BFS is optimal: e.g. 8 puzzle
  * **No** in general!
    * how about route finding problem ?

In [10]:
# formulate the route finding problem: Arad to Bucharest
r1 = RouteProblem('A', 'B', map=romania)
path_states(my_breadth_first_tree_search(r1))

['A', 'S', 'F', 'B']

BFS finds a solution [A, S, F, B]: with the fewest step but not lowest cost;
  * the path cost is 450

The optimal solution is [A, S, R, P, B]: 
  * more hops but with lower cost: **418**

**Time complexity**: depends on how many tree nodes been generated 
  * $O(b^d)$: b is the branching factor, d is the depth of the solution
  * $\underbrace{1}_{\text{root}}+ \underbrace{b}_{\text{depth 1}} + \underbrace{b^2}_{\text{depth 2}} + b^3 +\ldots, b^d = \frac{b^{d+1} -1}{b-1}=O(b^d)$
  

  <center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/grid6.png" width = "800" height="400"/></center>       

**Space complexity**: how much memory is needed 
  * still $O(b^d)$: b is the branching factor, d is the depth of the solution
    * popped node still needed later for solution building so cannot be freed up 
    * even popped nodes were not kept in memory, the size of **agenda** is still $O(b^{d})$
      * how many salmon nodes are there: roughly $b^{d+1} = b^3$: the size of agenda is the dominating term
    


# Avoid repeated states 

The BFS algorithm presented so far does a lot repetitive extension and goal checking 
  * e.g. **A** has been extended again at step 4; and it will be extended again later (A-D-A route)
  * same applies to a lot of others, B, D etc

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/grid5.png" width = "800" height="400"/></center>    

There are two* possible solutions 

**Option 1**: check cyclic path before pushing into agenda
  * we should reject [A,B,A] as it contains a cycle!
  * no extra storage cost
  * a lot of implementations trace back to some fixed level of ancesters, say 2 or 3 (not up to the root)

**Option 2**: maintain an **extended list** that keep track all extended nodes and do not add them to agenda if has been extended
  * after extend to [A,B,A]: it should not be added to the agenda
  * extra storage cost, which can grow very fast!

The AIAMA book has used a similar idea: `reached` node rather than `extended list`
  * the idea is similar, reached nodes keep track of nodes have been added to the agenda: 
    * i.e. keep **open list**'s history: agenda also called open list 
    * (open as yet to be explored)
  * extended list check nodes that have been popped and extended 
    * i.e. keep **closed list**'s history: extended list also called closed list 
    * (closed as been considered already)

Pseudo-code of BFS that avoids repeated nodes or cyclic paths

<!-- <div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
    
step 0: Initialize
   
    agenda = FIFOQueue();
    agenda.push([start]);
    [extended_list = {};]  // can be a set; optional
    
while agenda is not empty:
    
    1. path = agenda.pop()
    2. if is-path-to-goal(path, goal)
        return path
    3. otherwise extend path [if the ending node is not in the extended_list] 
        [add the ending node to extended_list]
        for each connected node (child node of the ending node of path)
            make a new_path that extends the connected node
            if new_path contains NO cylce 
                agenda.push(new_path)
fail    
</div> -->

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/bfspseudo.png" width = "800" height="400"/></center>  

The [...] parts are optional depends on whether extended list is in use or not.

In [11]:
def my_breadth_first_search(problem, use_extendedList = True):
    agenda = FIFOQueue([Node(problem.initial)])
#   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
                        agenda.appendleft(child)
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.appendleft(child)
    return failure

# Demonstration of complexity 


Check Panopto lecture pre-recording.

# A second searching algorithm: Depth-first-search (DFS)


We can obtain a second searching algorithm easily by changing the agenda's data structure


Only need to change one thing in the pseudo-code:

for **agenda** : 
$\text{Queue} \Rightarrow \text{Stack}$

  * stack is FILO or (LIFO)
  * or just add extended paths to the front of the queue (rather than back)
  
<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/29/Data_stack.svg/391px-Data_stack.svg.png" width = "800" height="400"/></center>  


Pseudo-code of DFS that checks cyclic 

<!-- <div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
    
step 0: Initialize
   
    agenda = LIFOStack();
    agenda.push([start]);
    [extended_list = {};]  // can be a set; optional
    
while agenda is not empty:
    
    1. path = agenda.pop()
    2. if is-path-to-goal(path, goal)
           return path
    3. otherwise extend path [if the ending node is not in extended_list]
       [add the ending node to extended_list]
       for each connected node (child node of the ending node of path)
           make a new_path that extends the connected node
           if new_path contains NO cylce
               agenda.push(new_path)
fail  
</div> -->

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/dfspseudo.png" width = "800" height="400"/></center>  

The [...] parts are optional depends on whether extended list is in use or not.

In [12]:
# Python implementation of DFS
def my_depth_first_search(problem):
    agenda = StackAgenda([Node(problem.initial)])
    while agenda:
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.append(child)
    return failure

# DFS vs BFS

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/bfsdfs.png" width = "1100" height="400"/></center>  



**Breadth first search (BFS)**
  * pushes uniformly into the tree; or floods the tree
    


**Depth first search (DFS)** is a very dedicated and determined solver
  * **dives into one branch of the tree** and do **not look back** until reaching a dead end (a node cannot be extended, e.g. 4) then it **backtrack** to previous unexplored junction
    * therefore, cyclic check (or extended list) is essential
    * otherwise, it expand and check A-B-A-B-A-B forever! not complete for sure 

### State space graph of the grid example
<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/grid.png" width = "300" height="400"/></center>  

### DFS search tree without cyclic path checking: A-B-A-B-A-B-...

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/treecycle.png" width = "500" height="400"/></center>  

The solution found by DFS for the grid problem: from A to G : A-B-C-F-E-D-G (definitely not optimal)

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/treedfs1.png" width = "600" height="400"/></center>  

# Performance analysis of DFS

**Complete**: if there is a solution, guaranteed to find it or not ?
  * **No!** 100% for DFS without cycle check !
  * still **No!** for problems with infinite state space
    * for example, an infinite grid, DFS might dive to the left forever but solution might on the r.h.s


**Optimal**: optimal solution found ?
  * **No** it dives into one branch not likely to be the best one 
  * e.g. Arad to Bucharest routing problem; the solution path found is much longer (costier as well)!

In [13]:
# formulate the route finding problem: Arad to Bucharest
r1 = RouteProblem('A', 'B', map=romania)
path_states(my_depth_first_search(r1))

['A', 'T', 'L', 'M', 'D', 'C', 'P', 'B']

**Time complexity**: still $O(b^d)$
  * still need to expand and check this number of nodes before finding the solution (assume the state space is finite)
  
  
**Space complexity**: linear $O(bd)$!
  * we only keep one branch: d levels of nodes, each level has roughly $b$ nodes 
  * after backtrack: searched branches can be discarded safely
    * goal state's solution only depends on the current branch
  * that's why `extended_list` is usually not used with DFS (cycle check is good enough)
    * if keeping the list, the space complexity is no longer linear !
    * no point using DFS anymore

# Demonstration 

Check Panopto lecture pre-recording.

# Summary

* Searching problem
* Searching algorithm
  * performance measure
  * BFS
  * DFS